In [ ]:
from tradingview_ta import TA_Handler, Interval

In [ ]:
def tradingview_recommendation(symbol, market):
    """
    Retrieves the tradingview recommendation for the given symbol and market.

    Parameters:
    symbol (str): The symbol for which the recommendation is being retrieved.
    market (str): The market in which the symbol is being traded.

    Returns:
    list: A list containing the summary, moving averages, and oscillators recommendations.
    """
    if market == "india":
        screener = "india"
        exchange = "NSE"
    else:
        screener = "america"
    try:
        if market == "america":
            try:
                symbol_handler = TA_Handler(
                    symbol=symbol,
                    screener=screener,
                    exchange="NYSE",
                    interval=Interval.INTERVAL_1_WEEK,
                )
                analysis = symbol_handler.get_analysis()
                summary = analysis.summary['RECOMMENDATION']
                moving_averages = analysis.moving_averages['RECOMMENDATION']
                oscillators = analysis.oscillators['RECOMMENDATION']
                if summary:  # Check if a result is obtained
                    return [summary, moving_averages, oscillators]
            except Exception as e:
                symbol_handler = TA_Handler(
                    symbol=symbol,
                    screener=screener,
                    exchange="NASDAQ",
                    interval=Interval.INTERVAL_1_WEEK,
                )
                analysis = symbol_handler.get_analysis()
                summary = analysis.summary['RECOMMENDATION']
                moving_averages = analysis.moving_averages['RECOMMENDATION']
                oscillators = analysis.oscillators['RECOMMENDATION']
                if summary:  # Check if a result is obtained
                    return [summary, moving_averages, oscillators]
        else:
            symbol_handler = TA_Handler(
                symbol=symbol,
                screener=screener,
                exchange=exchange,
                interval=Interval.INTERVAL_1_WEEK,
            )
            analysis = symbol_handler.get_analysis()
            summary = analysis.summary['RECOMMENDATION']
            moving_averages = analysis.moving_averages['RECOMMENDATION']
            oscillators = analysis.oscillators['RECOMMENDATION']
            return [summary, moving_averages, oscillators]
    except Exception as e:
        return['N_A', 'N_A', 'N_A']
    
symbols_list = ["TSLA", "MSFT", "AAPL", "AMZN", "GOOG", "META", "FB"]

for symbol in symbols_list:
    print(tradingview_recommendation(symbol, "america"))

In [ ]:
def tradingview_recommendation(symbol, market):
    """
    Retrieves the tradingview recommendation for the given symbol and market.

    Parameters:
    symbol (str): The symbol to retrieve the recommendation for.
    market (str): The market where the symbol is traded.

    Returns:
    list: A list containing the summary, moving averages, and oscillators recommendations.
    """
    screener = "india" if market == "india" else "america"
    exchange = "NSE" if market == "india" else "NYSE"

    try:
        handler = TA_Handler(
            symbol=symbol,
            screener=screener,
            exchange=exchange,
            interval=Interval.INTERVAL_1_WEEK,
        )
        analysis = handler.get_analysis()
        recommendation_summary = analysis.summary.get('RECOMMENDATION', 'N_A')
        moving_averages = analysis.moving_averages.get('RECOMMENDATION', 'N_A')
        oscillators = analysis.oscillators.get('RECOMMENDATION', 'N_A')
        return [recommendation_summary, moving_averages, oscillators]
    except Exception:
        return ['N_A', 'N_A', 'N_A']
    
symbols_list = ["TSLA", "MSFT", "AAPL", "AMZN", "GOOG", "META", "FB"]

for symbol in symbols_list:
    print(tradingview_recommendation(symbol, "america"))    

In [ ]:
def tradingview_recommendation(symbol, market):
    """
    Retrieves the tradingview recommendation for the given symbol and market.

    Parameters:
    symbol (str): The symbol to retrieve the recommendation for.
    market (str): The market where the symbol is traded.

    Returns:
    list: A list containing the summary, moving averages, and oscillators recommendations.
    """
    screener = "india" if market == "india" else "america"
    exchange = "NSE" if market == "india" else None

    try:
        handler = TA_Handler(
            symbol=symbol,
            screener=screener,
            exchange="NASDAQ",
            interval=Interval.INTERVAL_1_DAY,
        )
        analysis = handler.get_analysis()
        print("Test")
        recommendation_summary = analysis.summary.get('RECOMMENDATION', 'N_A')
        moving_averages = analysis.moving_averages.get('RECOMMENDATION', 'N_A')
        oscillators = analysis.oscillators.get('RECOMMENDATION', 'N_A')
        return [recommendation_summary, moving_averages, oscillators]
    except Exception as e:
        print(e)
        return ['N_A', 'N_A', 'N_A']
    
symbols_list = ["FSLR", "WBA", "KLAC", "MPWR", "LULU", "CSCO", "HOLX", "TRIP"]

for symbol in symbols_list:
    print(tradingview_recommendation(symbol, "america"))   

In [ ]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import numpy as np

data = yf.download("RCF.NS", start="2022-01-01", end="2024-04-24")
data = data.dropna()
data.tail()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Calculate trendline using linear regression
slope, intercept = np.polyfit(range(len(data)), data['Close'], 1)
trendline = intercept + slope * range(len(data))

# Detect breakout points
breakout_points = data[data['Close'] > trendline]

# Plot the data and trendline
plt.figure(figsize=(10, 6))
plt.plot(data.index, data['Close'], label='Close Price')
plt.plot(data.index, trendline, color='r', linestyle='--', label='Trendline')
plt.scatter(breakout_points.index, breakout_points['Close'], color='g', label='Breakout Points')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('Trendline Breakout')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Calculate historical volatility
data['Log Returns'] = np.log(data['Close'] / data['Close'].shift(1))
data['Volatility'] = data['Log Returns'].rolling(window=5).std() * np.sqrt(252)

# Define volatility threshold
volatility_threshold = 0.3

# Find volatility breakouts
volatility_breakouts = data[data['Volatility'] > volatility_threshold]

# Plot the data and volatility breakouts
plt.figure(figsize=(10, 6))
plt.plot(data.index, data['Close'], label='Close Price')
plt.scatter(volatility_breakouts.index, volatility_breakouts['Close'], color='r', label='Volatility Breakouts')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.title('Volatility Breakouts')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import pandas as pd

def get_ticker(company_name):
    url = "https://www.screener.in/company/" + company_name + "/consolidated/"
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"
    params = {"q": company_name, "quotes_count": 1, "country": "India"}
    res = pd.read_html(url)
    print(res)
    print("Test")

get_ticker("TATAMOTORS")

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def fetch_tables_as_dataframes(url):
    res = pd.read_html(url)
    return res

# Example usage
company_name = "TCS"
website_url = "https://www.screener.in/company/" + company_name
table_dfs = fetch_tables_as_dataframes(website_url)

# Print the dataframes
print("Dataframes:")
for df in table_dfs:
    # Make first column as index
    df.set_index(df.columns[0], inplace=True)
    # Remove unwanted rows
    df = df.dropna()
    # Print the dataframe
    print(df)

# Total number of tables
print("Total number of tables:", len(table_dfs))


In [1]:
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from time import sleep
import re
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore") 

In [2]:
symbol = "TCS.NS"
ticker_data = yf.Ticker(symbol)
ticker_data.info
# Calculate ROCE



{'address1': 'TCS House',
 'address2': 'Raveline Street Corner of Hazarimal Somani Marg Near Sterling Cinema, Fort',
 'city': 'Mumbai',
 'zip': '400 001',
 'country': 'India',
 'phone': '91 22 6778 9999',
 'fax': '91 22 6778 9000',
 'website': 'https://www.tcs.com',
 'industry': 'Information Technology Services',
 'industryKey': 'information-technology-services',
 'industryDisp': 'Information Technology Services',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Tata Consultancy Services Limited provides information technology (IT) and IT enabled services in the Americas, Europe, India, and internationally. It operates through Banking, Financial Services and Insurance; Manufacturing; Consumer Business; Communication, Media and Technology; Life Sciences and Healthcare; and Others segments. The company provides TCS ADD, a suite of technology platforms for clinical research and drug development; TCS BaNCS, a financial services plat

In [3]:
# Fetch stock data
def get_company_data(symbol):
    ticker_data = pd.Series()
    ticker_data_one = yf.Ticker(symbol).info
    ticker_data['Price'] = ticker_data_one['currentPrice']
    ticker_data['High'] = ticker_data_one['fiftyTwoWeekHigh']
    ticker_data['Low'] = ticker_data_one['fiftyTwoWeekLow']
    ticker_data['Market_Cap'] = ticker_data_one['marketCap']
    ticker_data['PE'] = ticker_data_one['trailingPE']
    ticker_data['Dividend'] = ticker_data_one['dividendYield']
    ticker_data['ROCE'] = ticker_data_one['returnOnCapitalEmployed']
    ticker_data['ROE'] = ticker_data_one['returnOnEquity']
    ticker_yearly_financials = yf.Ticker(symbol).financials
    ticker_quarterly_financials = yf.Ticker(symbol).quarterly_financials
    return ticker_data, ticker_yearly_financials, ticker_quarterly_financials

In [4]:
def check_current_below_high_threshold(current, high, threshold_percent = 10):
    below_threshold = False
    if ((current < high) & ((high-current)/high*100 > threshold_percent)):
        below_threshold = True
    return below_threshold 

In [5]:
def extract_pl(df):
    last_pl = df.loc['Gross Profit'].tolist()
    return last_pl

In [6]:
# PROFIT-LOSS STRATEGY - CONFIG
PROFIT_LOSS_YEARS = 3
SCREENER_ROW_PL = 'Net Profit' # or can be 'Profit before tax'
CURRENT_HIGH_THRESHOLD_PERCENT = 10

In [7]:
def apply_pl_strategy(symbol, current_price, high_price, yearly_financials, high_threshold_percent):
    
#     STRATEGY:
#     BUY recommendation if:
#         1. Profit/Loss for the company has been increasing consistently in the last few years.
#         2. Current market price is below 10% of 52-week high

    # SET DEFAULT TO STOCK AS NO-ACTION
    strategy_result = 'WAIT'
    try: 

        # CHECK IF REQUIRED VALUES COULD BE SCRAPED
        if (current_price is None or current_price == 0.0 or 
            high_price is None or high_price == 0.0):
            strategy_result = 'NOT FOUND'

        else:
            last_pl_list = extract_pl(yearly_financials, PROFIT_LOSS_YEARS)
            print(f'Profit/Loss for last {PROFIT_LOSS_YEARS} years:{last_pl_list}')
            print(f'Current Price:{current_price}, 52-week High:{high_price}, Threshold%: {high_threshold_percent}%')

            # CHECK IF PROFIT-LOSS IS CONSISTENTLY INCREASING
            if(last_pl_list == sorted(last_pl_list)):
                # IF YES, CHECK IF CURRENT MARKET VALUE IS NOT AT ALL TIME HIGH
                if check_current_below_high_threshold(current_price, high_price, high_threshold_percent):
                    # BUY RECOMMENDATION
                    strategy_result = 'BUY'
    except Exception as e:
        print(f"UNABLE TO APPLY PROFIT-LOSS STRATEGY ON {symbol}. Exception: {e}")

    return strategy_result

In [8]:
# Create dataframe
def create_dataframe(symbols):
    df = pd.DataFrame({'Symbol':symbols}, columns=['Symbol','Market_Cap','Price','High','Low','PE','ROE','ROCE','Dividend','STRATEGY_PL']).set_index('Symbol')
    for symbol in symbols:
        symbol_data, yearly_financials, quarterly_financials = get_company_data(symbol)
        strategy_result = apply_pl_strategy(symbol, symbol_data['Price'], symbol_data['High'], yearly_financials, CURRENT_HIGH_THRESHOLD_PERCENT)
    for index, value in symbol_data.iteritems():
        df[index][symbol] = value
        df['STRATEGY_PL'][symbol] = strategy_result
    return df

In [9]:
BANK_SYMOLS = ['HDFCBANK','ICICIBANK','AXISBANK','KOTAKBANK','SBIN','CANBK']
IT_SYMBOLS = ['INFY.NS', 'HCLTECH.NS', 'LTI.NS', 'TCS.NS','LTTS.NS', 'WIPRO.NS' ]
symbols = IT_SYMBOLS

In [10]:
final_df = create_dataframe(symbols)
plt.figure(figsize = (15,8))

mark_cap_sorted_df = final_df.sort_values(by='Market_Cap', ascending = False)
ax = sns.barplot(x=mark_cap_sorted_df.index, y=mark_cap_sorted_df['Market_Cap'], palette = 'bright')
ax.set(xlabel="Symbol", ylabel='Market Cap (in Cr)',title = f'MARKET CAPITALIZATION (in Crores)') 
sns.set(font_scale = 1.5)

#annotate axis = seaborn axis
for p in ax.patches:
     ax.annotate("%.0f Cr" % p.get_height(), (p.get_x() + p.get_width() / 2., p.get_height()),
         ha='center', va='center', fontsize=14,  xytext=(0, 10),
         textcoords='offset points')

KeyError: 'returnOnCapitalEmployed'